In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import joblib

# Загрузка модели и энкодеров
model = xgb.XGBClassifier()
model.load_model("FinalModel.json")

enc = joblib.load(r'c:/KR/encoder.joblib')  # Путь к файлу encoder.joblib
le = joblib.load(r'c:/KR/label_encoder.joblib') # Путь к файлу label_encoder.joblib

# Явно задаем selected
selected = ['Outstanding_Debt', 'Total_EMI_per_month', 'Interest_Rate',
            'Monthly_Inhand_Salary', 'Credit_Mix', 'Delay_from_due_date',
            'Amount_invested_monthly', 'Num_Credit_Inquiries',
            'Credit_History_Age', 'Num_Credit_Card', 'Payment_of_Min_Amount',
            'Changed_Credit_Limit']

# Запоминаем порядок столбцов
feature_order = ['Outstanding_Debt', 'Total_EMI_per_month', 'Interest_Rate', 'Monthly_Inhand_Salary', 'Credit_Mix', 'Delay_from_due_date', 'Amount_invested_monthly', 'Num_Credit_Inquiries', 'Credit_History_Age', 'Num_Credit_Card', 'Payment_of_Min_Amount', 'Changed_Credit_Limit']

# Создаем OrdinalEncoder для UI с категориями из обученного enc
enc_ui = OrdinalEncoder(categories=enc.categories, handle_unknown='use_encoded_value', unknown_value=-1)

# Перевод названий признаков для пользовательского интерфейса
feature_translations = {
    'Outstanding_Debt': 'Непогашенная задолженность',
    'Total_EMI_per_month': 'Общая сумма ежемесячных платежей по кредиту',
    'Interest_Rate': 'Процентная ставка',
    'Monthly_Inhand_Salary': 'Ежемесячный доход',
    'Credit_Mix': 'Качество кредитного портфеля',
    'Delay_from_due_date': 'Просрочка платежа (в днях)',
    'Amount_invested_monthly': 'Сумма ежемесячных инвестиций',
    'Num_Credit_Inquiries': 'Количество кредитных запросов',
    'Credit_History_Age': 'Возраст кредитной истории (в месяцах)',
    'Num_Credit_Card': 'Количество кредитных карт',
    'Payment_of_Min_Amount': 'Оплата минимального платежа',
    'Changed_Credit_Limit': 'Изменение кредитного лимита'
}

def predict_credit_score(button):
    input_data = {}
    for feature in selected:
        input_data[feature] = input_widgets[feature].value

    # Преобразование в DataFrame
    input_df = pd.DataFrame(input_data, index=[0])

    # Кодирование категориальных признаков с помощью enc_ui
    cat_features = ['Credit_Mix', 'Payment_of_Min_Amount']
    input_df[cat_features] = enc_ui.fit_transform(input_df[cat_features])

    # Приведение порядка признаков в соответствие
    input_df = input_df[feature_order]

    # Предсказание
    prediction = model.predict(input_df)
    # Преобразование числового предсказания в текстовую метку
    dict = {0:'Хороший',1:'Плохой',2:'Стандартный'}
    credit_score = dict[prediction[0]]

    print('Предсказанный кредитный рейтинг:', credit_score)  # Вывод текстовой метки

display(HTML("""
<style>
.widget-label {
    min-width: 300px !important;
}
</style>
"""))
input_widgets = {}

for feature in selected:
    if feature in ['Credit_Mix', 'Payment_of_Min_Amount']:
        # Dropdown для категориальных признаков
        # ВАЖНО: Замените эти значения на актуальные категории из вашего датасета
        unique_values = ['Good', 'Bad', 'Standard'] if feature == 'Credit_Mix' else ['Yes', 'No', 'NM']
        input_widgets[feature] = widgets.Dropdown(
            options=unique_values,
            description=feature_translations[feature] + ':',
            layout=widgets.Layout(width='400px')
        )
    else:
        # FloatText для числовых признаков
        # ВАЖНО: Замените эти значения на актуальные границы из вашего датасета
        min_val = 0
        max_val = 5000
        input_widgets[feature] = widgets.FloatText(
            description=feature_translations[feature],
            layout=widgets.Layout(width='400px')
        )

# Вывод виджетов
display(widgets.VBox(list(input_widgets.values())))

predict_button = widgets.Button(description="Предсказать кредитный рейтинг", layout=widgets.Layout(width='400px'))
predict_button.on_click(predict_credit_score)
display(predict_button)

Button(description='Предсказать кредитный рейтинг', layout=Layout(width='400px'), style=ButtonStyle())

Предсказанный кредитный рейтинг: Хороший
